In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# ----------------------------
# 1. Generate Synthetic Data
# ----------------------------

np.random.seed(42)
n_samples = 1000
img_size = 32
n_channels = 2  # e.g., NDVI and NIR

# Generate before and after images
def generate_image_pair(deforested):
    before = np.random.rand(img_size, img_size, n_channels)
    if deforested:
        after = before.copy()
        after[:, :, 0] *= np.random.uniform(0.4, 0.7)  # simulate NDVI drop
        after[:, :, 1] *= np.random.uniform(0.8, 1.0)  # minor NIR change
    else:
        after = before + np.random.normal(0, 0.01, before.shape)
    return np.concatenate([before, after], axis=-1)

X = np.zeros((n_samples, img_size, img_size, n_channels * 2))
y = np.zeros(n_samples)

for i in range(n_samples):
    is_deforested = i < n_samples // 2  # first half deforested
    X[i] = generate_image_pair(is_deforested)
    y[i] = int(is_deforested)

# ----------------------------
# 2. Train/Test Split
# ----------------------------

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ----------------------------
# 3. CNN Model
# ----------------------------

model = models.Sequential([
    layers.Conv2D(16, (3, 3), activation='relu', input_shape=(img_size, img_size, 4)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# ----------------------------
# 4. Train the Model
# ----------------------------

history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# ----------------------------
# 5. Evaluate Model
# ----------------------------

y_pred = (model.predict(X_test) > 0.5).astype("int32")
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# ----------------------------
# 6. Plot Training History
# ----------------------------

plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
